In [235]:
!pip install -q -U kaggle_environments

In [236]:
import numpy as np
import pandas as pd
import random


import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

In [237]:
%%writefile rock.py
#always "rock"
def rock(observation, configuration):
    return 0


Overwriting rock.py


In [238]:
%%writefile paper.py
#always "paper"
def paper(observation, configuration):
    return 1


Overwriting paper.py


In [239]:
%%writefile scissors.py
#always "scissors"
def scissors(observation, configuration):
    return 2

Overwriting scissors.py


In [240]:
%%writefile rand.py
#random
import random

def rand(observation, configuration):
    return random.randrange(0, 3)

Overwriting rand.py


In [241]:
%%writefile random_r_or_p.py
#random "rock" or "paper"
import random

def rand(observation, configuration):
    return random.randrange(0, 2)

Overwriting random_r_or_p.py


In [242]:
%%writefile random_s_or_p.py
#random "scissors" or "paper"
import random

def rand(observation, configuration):
    return random.randrange(1, 3)

Overwriting random_s_or_p.py


In [243]:
%%writefile random_s_or_r.py
#random "scissors" or "rock"
import random

def rand(observation, configuration):
    return random.choice([0, 2])

Overwriting random_s_or_r.py


In [244]:
%%writefile copy_opponent.py
import random

def copy_opponent(observation, configuration):
    if observation.step > 0:       
        return observation.lastOpponentAction
    else:  
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


In [245]:
%%writefile statistical.py
import random
action_histogram = {}


def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Overwriting statistical.py


In [246]:
evaluate(
    "rps", 
    ["scissors.py", "random_s_or_r.py"], 
    configuration={"episodeSteps": 100}
)

[[-47.0, 47.0]]

In [247]:
list_names = [
    "rock", 
    "paper", 
    "scissors",
    "rand",  
    "random_r_or_p", 
    "random_s_or_p", 
    "random_s_or_r", 
    "copy_opponent",
    "statistical", 
]

list_agents = [agent_name + ".py" for agent_name in list_names]

scores = 0

df_scores = pd.DataFrame(
    scores, 
    index=list_names, 
    columns=list_names,
)

In [248]:
for i in range(len(list_names)):
    for j in range(i + 1, len(list_names)):
        print(
            f"LOG: {list_names[i]} vs {list_names[j]}", 
            end="\r"
        )
        
        current_score = evaluate(
            "rps", 
            [list_agents[i], list_agents[j]], 
            configuration={"episodeSteps": 1000}
        )
        
        df_scores[list_names[i]][list_names[j]] = current_score[0][0]
        df_scores[list_names[j]][list_names[i]] = current_score[0][1]

df_scores

,rock,paper,scissors,rand,random_r_or_p,random_s_or_p,random_s_or_r,copy_opponent,statistical
rock,0,999,-999,0,499,21,-480,0,998
paper,-999,0,999,-44,-496,511,-27,0,997
scissors,999,-999,0,-44,0,-510,482,0,999
rand,0,44,44,0,-27,26,-28,0,0
random_r_or_p,-499,496,0,27,0,210,-245,0,78
random_s_or_p,-21,-511,510,-26,-210,0,217,0,530
random_s_or_r,480,27,-482,28,245,-217,0,0,393
copy_opponent,0,0,0,0,0,0,0,0,201
statistical,-998,-997,-999,0,-78,-530,-393,-201,0
